In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=3b28527297616de23aa24c37d400daec82f1e592794c978f4a03546c5d4e0829
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import surprise
from surprise import Dataset, Reader, SVD, KNNWithMeans
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

In [ ]:
dataset = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
# Виведення перших 5 рядків датасету з поясненнями
print("Перші 5 рядків датасету:")
for row in dataset.raw_ratings[:5]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")

Перші 5 рядків датасету:
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)


In [ ]:
# Розбиття на тренувальний і тестовий набори
train_set, test_set = train_test_split(dataset, test_size=0.25)

# Параметри для підбору для алгоритму SVD
param_grid_svd = {
    'n_factors': [60, 90],
    'lr_all': [0.004, 0.008],
    'reg_all': [0.03, 0.07]
}

In [ ]:
# Налаштування крос-валідації з GridSearchCV
svd_search = GridSearchCV(SVD, param_grid_svd, measures=['mae'], cv=3)
svd_search.fit(dataset)

In [ ]:
# Параметри для алгоритму KNNWithMeans
param_grid_knn = {
    'k': [10, 20],
    'min_k': [2, 4],
    'sim_options': {
        'name': ['pearson_baseline', 'msd'],
        'user_based': [False, True]
    }
}

knn_search = GridSearchCV(KNNWithMeans, param_grid_knn, measures=['mae'], cv=3)
knn_search.fit(dataset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Comput

In [ ]:
# Отримання найкращих моделей
best_svd_model = svd_search.best_estimator['mae']
best_knn_model = knn_search.best_estimator['mae']

In [ ]:
# Порівняння моделей за MAE
print(f"Найкращий MAE для SVD: {svd_search.best_score['mae']}")
print(f"Найкращий MAE для KNNWithMeans: {knn_search.best_score['mae']}")

Найкращий MAE для SVD: 0.7352533763088802
Найкращий MAE для KNNWithMeans: 0.7305858153596024


In [ ]:
# Вибір найкращої моделі
selected_model = best_svd_model if svd_search.best_score['mae'] < knn_search.best_score['mae'] else best_knn_model
selected_model_name = "SVD" if selected_model == best_svd_model else "KNNWithMeans"
print(f"Обрано найкращий алгоритм: {selected_model_name}")

Обрано найкращий алгоритм: KNNWithMeans


2 ЗАВДАННЯ

In [ ]:
# Навчання обраної моделі на всьому датасеті
full_trainset = dataset.build_full_trainset()
selected_model.fit(full_trainset)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
# Вибір конкретного користувача для рекомендацій
user_id = '85'  # Замінити на ID користувача


In [ ]:
# Отримання списку всіх фільмів
all_items = full_trainset.all_items()
all_movie_ids = [full_trainset.to_raw_iid(item_id) for item_id in all_items]

In [ ]:
# Прогнозування оцінок для всіх фільмів, які користувач ще не оцінив
predictions = [selected_model.predict(user_id, movie_id) for movie_id in all_movie_ids]
top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

In [ ]:
# Виведення топ-10 рекомендацій
print(f"Топ-10 рекомендацій для користувача {user_id}:")
for pred in top_recommendations:
    print(f"Фільм ID: {pred.iid}, прогнозований рейтинг: {pred.est:.2f}")

Топ-10 рекомендацій для користувача 85:
Фільм ID: 814, прогнозований рейтинг: 5.00
Фільм ID: 1536, прогнозований рейтинг: 5.00
Фільм ID: 1599, прогнозований рейтинг: 5.00
Фільм ID: 1653, прогнозований рейтинг: 5.00
Фільм ID: 1122, прогнозований рейтинг: 5.00
Фільм ID: 1201, прогнозований рейтинг: 5.00
Фільм ID: 1449, прогнозований рейтинг: 4.80
Фільм ID: 1467, прогнозований рейтинг: 4.80
Фільм ID: 134, прогнозований рейтинг: 4.75
Фільм ID: 1293, прогнозований рейтинг: 4.70


У цій лабораторній  було використано датасет "ml-100k", що містить рецензії на фільми. На основі цього набору даних була побудована система, яка здатна прогнозувати оцінки фільмів для конкретних користувачів. Було отримано рекомендації для обраного користувача, що демонструє практичну роботу системи рекомендацій.